### 1. Imports 

In [ ]:
import os
import random
import time
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch
import torchvision
import torch.nn as nn
from torchvision.transforms import ToPILImage
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [ ]:
def random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
    
random_seed(1702)

### 2. Configuration

In [ ]:
width = 704
height = 520
cell_dict = {"astro": 1, "cort": 2, "shsy5y": 3}
num_box_detections = 600
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
n_epochs = 10
batch_size = 2
learning_rate = 0.001
momentum = 0.9
weight_decay = 0.0005 

NORMALIZE = True
RESNET_MEAN = (0.485, 0.456, 0.406)
RESNET_STD = (0.229, 0.224, 0.225)

### 3. Helpers 

In [ ]:
# ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return
    color: color for the mask
    Returns numpy array (mask)

    '''
    s = mask_rle.split()

    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    if len(shape)==3:
        img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    else:
        img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for start, end in zip(starts, ends):
        img[start : end] = color

    return img.reshape(shape)


def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))


def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

def combine_masks(masks, mask_threshold):
    """
    combine masks into one image
    """
    maskimg = np.zeros((height, width))
    # print(len(masks.shape), masks.shape)
    for m, mask in enumerate(masks,1):
        maskimg[mask>mask_threshold] = m
    return maskimg


def get_filtered_masks(pred):
    """
    filter masks using MIN_SCORE for mask and MAX_THRESHOLD for pixels
    """
    use_masks = []   
    for i, mask in enumerate(pred["masks"]):

        # Filter-out low-scoring results. Not tried yet.
        scr = pred["scores"][i].cpu().item()
        label = pred["labels"][i].cpu().item()
        if scr > min_score_dict[label]:
            mask = mask.cpu().numpy().squeeze()
            # Keep only highly likely pixels
            binary_mask = mask > mask_threshold_dict[label]
            binary_mask = remove_overlapping_pixels(binary_mask, use_masks)
            use_masks.append(binary_mask)

    return use_masks

In [ ]:
# Ref: https://www.kaggle.com/theoviel/competition-metric-map-iou
        
def compute_iou(labels, y_pred, verbose=0):
    """
    Computes the IoU for instance labels and predictions.

    Args:
        labels (np array): Labels.
        y_pred (np array): predictions

    Returns:
        np array: IoU matrix, of size true_objects x pred_objects.
    """

    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    if verbose:
        print("Number of true objects: {}".format(true_objects))
        print("Number of predicted objects: {}".format(pred_objects))

    # Compute intersection between all objects
    intersection = np.histogram2d(
        labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects)
    )[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins=true_objects)[0]
    area_pred = np.histogram(y_pred, bins=pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    intersection = intersection[1:, 1:] # exclude background
    union = union[1:, 1:]
    union[union == 0] = 1e-9
    iou = intersection / union
    
    return iou  

def precision_at(threshold, iou):
    """
    Computes the precision at a given threshold.

    Args:
        threshold (float): Threshold.
        iou (np array): IoU matrix.

    Returns:
        int: Number of true positives,
        int: Number of false positives,
        int: Number of false negatives.
    """
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    return tp, fp, fn

def iou_map(truths, preds, verbose=0):
    """
    Computes the metric for the competition.
    Masks contain the segmented pixels where each object has one value associated,
    and 0 is the background.

    Args:
        truths (list of masks): Ground truths.
        preds (list of masks): Predictions.
        verbose (int, optional): Whether to print infos. Defaults to 0.

    Returns:
        float: mAP.
    """
    ious = [compute_iou(truth, pred, verbose) for truth, pred in zip(truths, preds)]

    if verbose:
        print("Thresh\tTP\tFP\tFN\tPrec.")

    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tps, fps, fns = 0, 0, 0
        for iou in ious:
            tp, fp, fn = precision_at(t, iou)
            tps += tp
            fps += fp
            fns += fn

        p = tps / (tps + fps + fns)
        prec.append(p)

        if verbose:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tps, fps, fns, p))

    if verbose:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

    return np.mean(prec)


def get_score(ds, mdl):
    """
    Get average IOU mAP score for a dataset
    """
    mdl.eval()
    iouscore = 0
    for i in tqdm(range(len(ds))):
        img, targets = ds[i]
        with torch.no_grad():
            result = mdl([img.to(DEVICE)])[0]
            
        masks = combine_masks(targets['masks'], 0.5)
        labels = pd.Series(result['labels'].cpu().numpy()).value_counts()

        mask_threshold = mask_threshold_dict[labels.sort_values().index[-1]]
        pred_masks = combine_masks(get_filtered_masks(result), mask_threshold)
        iouscore += iou_map([masks],[pred_masks])
    return iouscore / len(ds)

### 4. Dataloader

In [ ]:
# Ref https://www.kaggle.com/abhishek/maskrcnn-utils

class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class VerticalFlip:
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-2)
            bbox = target["boxes"]
            bbox[:, [1, 3]] = height - bbox[:, [3, 1]]
            target["boxes"] = bbox
            target["masks"] = target["masks"].flip(-2)
        return image, target

class HorizontalFlip:
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-1)
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
            target["boxes"] = bbox
            target["masks"] = target["masks"].flip(-1)
        return image, target

class Normalize:
    def __call__(self, image, target):
        image = F.normalize(image, RESNET_MEAN, RESNET_STD)
        return image, target

class ToTensor:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target
    

def get_transform(train):
    transforms = [ToTensor()]
    if NORMALIZE:
        transforms.append(Normalize())
    
    # Data augmentation for train
    if train: 
        transforms.append(HorizontalFlip(0.5))
        transforms.append(VerticalFlip(0.5))

    return Compose(transforms)

In [ ]:
class SatoriusDataset(Dataset): 
    def __init__(self, csv_file, image_dir, transform=None, resize=None):
        self.transform = transform
        self.csv = csv_file 
        self.resize = True if resize else False 
        
        # resize is the factor, e.g 1.2, 1.3, ... 
        if self.resize: 
            self.height = int(height * resize)
            self.width = int(width * resize)
        else: 
            self.height = height
            self.width = width
        
        # collections.defaultdict returns an empty dict when the key is not found
        self.image_info = collections.defaultdict(dict)
        
        # group by id then cell, and get the annotations 
        csv_groupby = self.csv.groupby(['id', 'cell_type'])['annotation'].agg(lambda x: list(x)).reset_index()
        
        
        # image_info is the replacement of csv 
        # use image_info to iterate
        for index, row in csv_groupby.iterrows(): 
            self.image_info[index] = { 
                'image_id': row['id'],
                'image_path': os.path.join(image_dir, row['id'] + '.png'),
                'annotations': row['annotation'],
                'cell_type': cell_dict[row['cell_type']],
            }        
        
    def __len__(self): 
        return len(self.image_info)

    def get_boxes(self, mask):
        # the size of the mask is [w, h] and the values are True and False
        # using np.where will return the location of the True value in the mask 
        mask = np.where(mask)

        # locations of the bounding box 
        x_min = np.min(mask[1])
        x_max = np.max(mask[1])
        y_min = np.min(mask[0])
        y_max = np.max(mask[0])

        return [x_min, y_min, x_max, y_max]

    def __getitem__(self, idx): 
        item = self.image_info[idx]

        image = cv2.imread(item['image_path'], cv2.IMREAD_COLOR)        
        if self.resize: 
            image = cv2.resize(image, (self.width, self.height))
        
        # height goes before width because of rle_decode()
        masks = np.zeros((len(item['annotations']), self.height, self.width), dtype=np.uint8)
        boxes = []

        for i, mask in enumerate(item['annotations']): 
            mask = rle_decode(mask, (height, width))

            if self.resize: 
                mask = cv2.resize(mask, (self.width, self.height))
                
            # the mask only contains true and false, which are 1 and 0
            mask = np.array(mask) > 0
            masks[i,:,:] = mask
            boxes.append(self.get_boxes(mask))

        labels = [item['cell_type'] for _ in range(len(item['annotations']))]

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        # targets must have for the mask-rcnn
        target = {
            "boxes": boxes,
            "labels": labels,
            "masks": masks,
        }

        if self.transform: 
            image, target = self.transform(image, target)

        return image, target

In [ ]:
# test the dataset

test_dataset_csv = pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv")
test_dataset = SatoriusDataset(test_dataset_csv, "../input/sartorius-cell-instance-segmentation/train")

test = test_dataset.__getitem__(1)
img = test[0]
target = test[1]

boxes = target['boxes']
masks = target['masks']
labels = target['labels']

print(f"img  :  {img.shape}")
print(f"boxes:  {boxes.shape}")
print(f"labels: {labels.shape}")
print(f"masks:  {masks.shape}")

print("labels:   ", labels.unique())

_, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))
ax[0].imshow(img)
ax[1].imshow(combine_masks(masks, 0))

for box in boxes:
    # box[1] - box[3] because y-axis is reversed
    ax[1].add_patch(patches.Rectangle((box[0], box[3]), box[2] - box[0], box[1] - box[3], linewidth=1, edgecolor='r', facecolor='none'))
    
plt.axis(False)
plt.show()

### 5. Model 

In [ ]:
class MaskRCNN(nn.Module):
    def __init__(self, num_classes, pretrained_model=None):
        super().__init__()
        
        self.pretrained_model = pretrained_model
        
        if NORMALIZE: 
            self.model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True,
                                                                      box_detections_per_img=num_box_detections, 
                                                                      image_mean=RESNET_MEAN, 
                                                                      image_std=RESNET_STD)
        else:
            self.model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True,
                                                                      box_detections_per_img=num_box_detections)

        hidden_layer = 256
        
        # get the input features of the box_predictor 
        # replace the pretrained box_predictor layer with new FastRCNN
        in_features = self.model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes+1)
        
        # get the input features of the mask_predictor 
        # replace the pretrained mask_predictor layer with new MaskRCNN
        in_features_mask = self.model.roi_heads.mask_predictor.conv5_mask.in_channels
        self.model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes+1)
                
    def forward(self, x, y=None): 
        # in the training mode, the model needs both x and y (y is the target)
        # in the val mode, the model only needs x
        if y: 
            return self.model(x, y)
        
        return self.model(x)
    
#     def init_model(self):
#         if self.pretrained_model:
#             self.model.load_state_dict(self.pretrained_model, map_location=device)
        
#         print("Loaded pretrained weights")

### 6. Training

In [ ]:
class Trainer:
    def __init__(self, model, optimizer, device, scheduler):
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        
        self.train_loss_mask = []
        self.train_loss_classifier = []
        self.train_loss = []
        self.val_loss_mask = []
        self.val_loss_classifier = []
        self.val_loss = []
        
    def fit(self, train_loader, val_loader, n_epochs):
        for epoch in range(n_epochs): 
            
            # train and val
            self.train_epoch(train_loader)            
            self.val_epoch(val_loader)
            
            if self.scheduler: 
                self.scheduler.step() 
            
            print(f"epoch {epoch + 1} train:  mask loss : {self.train_loss_mask[-1]:3f}, classifier loss:  {self.train_loss_classifier[-1]:3f}")
            print(f"epoch {epoch + 1} val:    mask loss : {self.val_loss_mask[-1]:3f},   classifier loss: {self.val_loss_classifier[-1]:3f}")
            print(f"epoch {epoch + 1}:        train loss: {self.train_loss[-1]:3f},      val_loss       : {self.val_loss[-1]:3f}")

            print('save model ...')
            torch.save(self.model.state_dict(), f'model-e{epoch}.pt')

            print('\n\n')
                                
    def train_epoch(self, train_loader):
        self.model.train()
        self.train_loss_mask.append(0)
        self.train_loss_classifier.append(0)
        self.train_loss.append(0)

        for i, data in tqdm(enumerate(train_loader, 0)): 
            x, y = data
            x = list(img.to(self.device) for img in x)
            y = [{k: v.to(self.device) for k, v in t.items()} for t in y]
            
            self.optimizer.zero_grad()
            
            # in training mode, the model return the loss
            # in val mode, the model return the predictions
            losses = self.model(x, y)
            loss = sum(loss for loss in losses.values())
            loss.backward()
            self.optimizer.step()
            
            self.train_loss_mask[-1] += losses['loss_mask'].item()
            self.train_loss_classifier[-1] += losses['loss_classifier'].item()
            self.train_loss[-1] += loss.item()
        
        # gets the average
        self.train_loss_mask[-1] /= len(train_loader)
        self.train_loss_classifier[-1] /= len(train_loader)
        self.train_loss[-1] /= len(train_loader)
        
    def val_epoch(self, val_loader):
        self.val_loss_mask.append(0)
        self.val_loss_classifier.append(0)
        self.val_loss.append(0)

        for i, data in tqdm(enumerate(val_loader, 0)): 
            with torch.no_grad():
                x, y = data 
                x = list(img.to(self.device) for img in x)
                y = [{k: v.to(self.device) for k, v in t.items()} for t in y]
                
                # in training mode, the model return the loss
                # in val mode, the model return the predictions
                losses = model(x, y)
                loss = sum(loss for loss in losses.values())
                
                self.val_loss_mask[-1] += losses['loss_mask'].item()
                self.val_loss_classifier[-1] += losses['loss_classifier'].item()
                self.val_loss[-1] += loss.item()

        # gets the average
        self.val_loss_mask[-1] /= len(val_loader)
        self.val_loss_classifier[-1] /= len(val_loader)
        self.val_loss[-1] /= len(val_loader)

In [ ]:
transform = get_transform(False)

train_csv = pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv")

# the layout of the df_images would be: id, cell_type, annotation
# there would be one row for one image because each image only contains one cell_type
df_images = train_csv.groupby(["id", "cell_type"]).agg({'annotation': 'count'}).sort_values("annotation", ascending=False).reset_index()

train_df, test_df = train_test_split(df_images, stratify=df_images['cell_type'], 
                                                  test_size=0.2,
                                                  random_state=1702)

# dataloader for training
train_df = train_csv[train_csv['id'].isin(train_df['id'])]
train_dataset = SatoriusDataset(train_df, "../input/sartorius-cell-instance-segmentation/train", transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True,
                              num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

# dataloader for validation
test_df = train_csv[train_csv['id'].isin(test_df['id'])]
test_dataset = SatoriusDataset(test_df, "../input/sartorius-cell-instance-segmentation/train", transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, pin_memory=True,
                             num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

In [ ]:
model = MaskRCNN(len(cell_dict))
model.to(device)

for param in model.parameters():
    param.requires_grad = True
    
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

trainer = Trainer(model, optimizer, device, lr_scheduler)
trainer.fit(train_dataloader, test_dataloader, n_epochs)

In [ ]:
# to-do gets the best model

In [53]:
# to-do: include the iou score for the prediction's image

test_dataset_temp = SatoriusDataset(test_df, "../input/sartorius-cell-instance-segmentation/train")

best_model = "../input/satorius-models/model-e9.pt"
num_rows = 3
fig, ax = plt.subplots(num_rows * batch_size, 3, figsize=(15, 20))

model = MaskRCNN(len(cell_dict))
model.to(device)

checkpoint = torch.load(best_model)
model.load_state_dict(checkpoint)
model.eval()

for i, data in enumerate(test_dataloader): 
    x, y = data
    x = list(img.to(device) for img in x)
    
    with torch.no_grad():
            predictions = model(x, None)
    
    for j in range(batch_size):
        original_img = test_dataset_temp.__getitem__(i * 2 + j)[0]
        
        ax[i * 2 + j, 0].imshow(original_img)
        ax[i * 2 + j, 1].imshow(combine_masks(y[j]["masks"], 0))
        ax[i * 2 + j, 2].imshow(combine_masks(predictions[j]["masks"].squeeze().cpu().numpy(), 0))
        
        ax[i * 2 + j, 0].axis('off')
        ax[i * 2 + j, 1].axis('off')
        ax[i * 2 + j, 2].axis('off')

        ax[i * 2 + j, 1].title.set_text(f'num_cells: {len(y[j]["masks"])}')
        ax[i * 2 + j, 2].title.set_text(f'num_cells: {len(predictions[j]["masks"].squeeze().cpu().numpy())}')
        
    if i == num_rows - 1: 
        break

In [ ]:
# to-do: predict